In [1]:
import pandas as pd
import numpy as np
#from sklearn.preprocessing import OneHotEncoder
#import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train1  = pd.read_csv('../Data/train_2.csv')

In [4]:
train1

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2017-09-01,2017-09-02,2017-09-03,2017-09-04,2017-09-05,2017-09-06,2017-09-07,2017-09-08,2017-09-09,2017-09-10
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,19.0,33.0,33.0,18.0,16.0,27.0,29.0,23.0,54.0,38.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,32.0,30.0,11.0,19.0,54.0,25.0,26.0,23.0,13.0,81.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,6.0,6.0,7.0,2.0,4.0,7.0,3.0,4.0,7.0,6.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,7.0,19.0,19.0,9.0,6.0,16.0,19.0,30.0,38.0,4.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.0,16.0,19.0,9.0,20.0,23.0,28.0,14.0,8.0,7.0
5,5566_zh.wikipedia.org_all-access_spider,12.0,7.0,4.0,5.0,20.0,8.0,5.0,17.0,24.0,...,13.0,13.0,45.0,4.0,13.0,20.0,18.0,17.0,14.0,11.0
6,91Days_zh.wikipedia.org_all-access_spider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.0,8.0,5.0,7.0,8.0,10.0,8.0,5.0,3.0,5.0
7,A'N'D_zh.wikipedia.org_all-access_spider,118.0,26.0,30.0,24.0,29.0,127.0,53.0,37.0,20.0,...,74.0,39.0,11.0,55.0,71.0,44.0,25.0,39.0,25.0,50.0
8,AKB48_zh.wikipedia.org_all-access_spider,5.0,23.0,14.0,12.0,9.0,9.0,35.0,15.0,14.0,...,53.0,107.0,63.0,42.0,24.0,44.0,33.0,52.0,21.0,48.0
9,ASCII_zh.wikipedia.org_all-access_spider,6.0,3.0,5.0,12.0,6.0,5.0,4.0,13.0,9.0,...,20.0,16.0,22.0,19.0,21.0,32.0,34.0,29.0,23.0,25.0


In [3]:
page = train1.iloc[:,0]

In [4]:
value = train1.iloc[:,1:].values

In [5]:
missing = np.isnan(value)

In [8]:
# ind2 is the index after which there is no missing
ind2 = missing.shape[1] - missing[:,::-1].argmax(1)
ind2 = np.where(np.all(np.logical_not(missing),1),0,ind2)

In [9]:
# exclude lower std sequence
ind1 = np.nanstd(value[:,:-60],1)
long_index = np.logical_and(ind2<803-60-60-430,ind1>1e-3)
short_index = np.logical_not(long_index)

In [10]:
# only long use RNN+Conv, short will use simply model
long_value = value[long_index]
long_page = page.loc[long_index]
short_value = value[short_index]
short_page = page.loc[short_index]

In [11]:
# after start_index, there is no missing, so safe for modeling
start_index = ind2[long_index]

In [12]:
# leave out last 60 for validations
stds = np.nanstd(long_value[:,:-60],1,keepdims=True)
means = np.nanmean(long_value[:,:-60],1,keepdims=True)

In [13]:
long_normal_value = (long_value - means)/stds

In [14]:
np.save('../Data/long_page.npy',long_page)
np.save('../Data/long_value.npy',long_value.astype(np.float32))
np.save('../Data/short_page.npy',short_page)
np.save('../Data/short_value.npy',short_value)
np.save('../Data/start_index.npy',start_index)
np.save('../Data/stds.npy',stds.astype(np.float32))
np.save('../Data/means.npy',means.astype(np.float32))
np.save('../Data/long_normal_value.npy',long_normal_value.astype(np.float32))

Generate Page features

In [10]:
a = train1.Page.str.split('mediawiki.org|wikimedia.org|wikipedia.org',expand=True)
b = np.where(a.loc[:,2].isnull(),a.loc[:,1],a.loc[:,2])
_,index_array = np.unique(b,return_inverse=True)
website_array = [train1.Page.str.contains(web).values.astype(np.int8) \
                 for web in ['mediawiki.org','wikimedia.org','wikipedia.org']] # website
lang_array = train1.Page.str.extract('([a-z]{2}).wikipedia').fillna('nan').values.flatten()
_,lang_array = np.unique(lang_array,return_inverse=True) # language

In [25]:
Page_X = \
np.concatenate([OneHotEncoder(dtype=np.float32,sparse=False,categories='auto').fit_transform(lang_array[:,np.newaxis]),\
                OneHotEncoder(dtype=np.float32,sparse=False,categories='auto').fit_transform(index_array[:,np.newaxis]),\
                np.stack(website_array,1)],1)

In [33]:
np.save('../Data/Page_X.npy',Page_X)

Save the rest

In [43]:
train1 = train1.iloc[:,1:]
IsMissing = train1.isnull().values
value = train1.T.fillna(pd.Series(np.nanmean(train1.values,1),train1.index),axis=0)
value = value.T.values.astype(np.float32)
start_index = np.argmin(IsMissing,1)
date_index=pd.to_datetime(train1.columns).to_series().reset_index(drop=True)

In [111]:
np.save('../Data/value.npy',value)
np.save('../Data/IsMissing.npy',IsMissing)
np.save('../Data/start_index.npy',start_index)

In [112]:
date_index.to_pickle('../Data/date_index')